In [1]:
import os
os.environ["SPARK_HOME"] = "/Users/zouhairhajji/Documents/dev/spark-2.4.0-bin-hadoop2.7"

In [2]:
from pyspark.sql import SparkSession
import csv
import numpy as np
import random
from math import sqrt

spark = SparkSession.builder \
        .master('local[*]')  \
        .enableHiveSupport() \
        .getOrCreate()

sc = spark.sparkContext

In [3]:

def loadData(namefile) :
    textfile = sc.textFile(namefile)    \
            .zipWithIndex()   \
            .map(lambda x: (x[1], x[0].split(',')  ))  \
            .map(lambda x: (x[0], [float(i) for i in x[1]]  ))
            
    return textfile

def dimension(rdd_load_data) :
    return rdd_load_data.map(lambda x: len(x[1])).reduce(lambda x, y: max(x, y)) 






def closestcentroide(list1):
    cluster = list1[0][0]
    min_dist = list1[0][1]
    for elem in list1:
        if elem[1] < min_dist:
            cluster  = elem[0]
            min_dist = elem[1]
    return (cluster, min_dist)

def computeCentroids(rdd,findcentroid):

    union = findcentroid.join(rdd)
    centroid_point = union.map(lambda x: (x[1][0][0],x[1][1][:-1])) 
    numpoints = centroid_point.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y) 
    sum = centroid_point.reduceByKey(lambda x,y: [x[i]+y[i] for i in range(len(x))]) 
    union2 = sum.join(numpoints)
    newcentroids = union2.map(lambda x : (x[0], mean(x[1][0],x[1][1]))) 

    return newcentroids  


def calculate_distance(x, y):
    s = sqrt(sum([ (a - b)**2 for a,b in zip(x,y)]))
    dis = float('%.3f'%(s))
    return dis



def initCentroids(rdd_flat_data, k):
    seed = random.randint(0, 100)
    
    centroids = rdd_flat_data.takeSample(False, k, seed=seed)
    return  sc.parallelize(centroids)   \
                .zipWithIndex()         \
                .map(lambda x: (x[1], x[0][1])) 

    
def computeIntraClusterDistance(rdd, centroids):
    initdistance = assignToCluster(rdd,centroids) 
    totaldistance = initdistance.map(lambda x: (1,x[1][1])).reduceByKey(lambda x,y: x+y)
    return totaldistance
    
def mean(x,n):
    return [x[i]/n for i in range(len(x))]

flat_data = loadData('iris_input.data')
 
initCentroids(flat_data, 3).collect()




[(0, [5.7, 2.5, 5.0, 2.0]),
 (1, [5.1, 3.7, 1.5, 0.4]),
 (2, [5.5, 2.3, 4.0, 1.3])]

In [8]:
def calculate_distance(x, y):
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(np.sum(np.square( x-y )))  
    
    
def assignToCluster(rdd, centroids):
    cartes = rdd.cartesian(centroids) 
    
    cartes = cartes.map(lambda x: (x[0][0], (x[1][0], calculate_distance(x[1][1], x[0][1]))  ) )
    cartes = cartes.groupByKey().mapValues(closestcentroide)
    return cartes


def closestcentroide(y):
    y = list(y)
    closest = y[0]
    
    for couple in y:
        if(closest[1] > couple[1]): 
            closest = couple
            
    return  closest

def calculate_new_coordinate(x): 
    mean = lambda x: sum(x) / len(x)
    
    return [ round(mean(x), 2) for x in zip(*x)]  

def computeCentroids(rdd, assigned_centroids): 
     return assigned_centroids   \
                .join(rdd)       \
                .map(lambda x: (x[1][0][0], x[1][1]) )       \
                .groupByKey()       \
                .mapValues(calculate_new_coordinate)

def computeIntraClusterDistance(rdd, centroids):
    initdistance = assignToCluster(rdd, centroids) 
    totaldistance = initdistance.map(lambda x: (1, x[1][1]) ).reduceByKey(lambda x,y: x+y)
    return totaldistance   

def kmeans(filename, k, m, d):
    flat_data = loadData(filename)
    dim = dimension(flat_data)
    
    centroids = initCentroids(flat_data, k)
    
    
    
    #assign every point to a centroids
    assigned_points = assignToCluster(flat_data, centroids)
    
    #compute centroids
    old_centroids = computeCentroids(flat_data, assigned_points)
    
    converged = False
    step_count = 0
    step_max = 10
    
    while not converged  and step_count < step_max:
        step_count += 1
        print('running step ', step_count)
        
        
        assigned_points = assignToCluster(flat_data, old_centroids) 
        
        new_centroids   = computeCentroids(flat_data, assigned_points)
        distance_intracluster = computeIntraClusterDistance(flat_data, new_centroids)
        
        
        centroid_changed = old_centroids.join(new_centroids) \
            .map(lambda x: np.sqrt(np.sum(np.square(np.array(x[1][0]) - np.array(x[1][1])))) ) \
            .reduce(lambda x, y: x+y) 
        
        print('    ->  ' ,  centroid_changed)
        
        
         
        if centroid_changed == 0:
            converged = True 
        old_centroids = sc.parallelize(new_centroids.collect()) 
            
        
    return (step_count, distance_intracluster.take(1), assigned_points, old_centroids  ) 
     
    

step_count, distance, points, centroids= kmeans('iris_input.data', 4, 4, 10)
print("distance : " , distance)
print("step_coup : " , step_count)


running step  1
    ->   3.0925925698730383
running step  2
    ->   0.9849446161390345
running step  3
    ->   0.09082776850515689
running step  4
    ->   0.050952410538477044
running step  5
    ->   0.0
distance :  [(1, 84.66865903654964)]
step_coup :  5


In [11]:
points.map(lambda x: (x[1][0], x[0]) ).groupByKey().mapValues(lambda x: len(list(x))).collect()

[(0, 47), (1, 23), (2, 50), (3, 30)]

#  ml lib


In [12]:

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler

In [15]:
def kmeans_mllib(filename, k):
    flat_data = loadData(filename).map(lambda x: x[1])  
    
    features = ["c1", "c2", "c3", "c4"]
    dataset = spark.createDataFrame(flat_data, features)

     
    
    transformed_dataset = VectorAssembler(inputCols=features, outputCol="features").transform(dataset) 
    
    
    kmeans = KMeans(k=k, seed=1) 
    model = kmeans.fit(transformed_dataset.select('features'))
    
    return model.transform(transformed_dataset) 
    
kmeans_mllib = kmeans_mllib('iris_input.data', 4)

 

In [21]:
kmeans_mllib.groupBy('prediction').count().show()

points.map(lambda x: (x[1][0], x[0]) ).groupByKey().mapValues(lambda x: len(list(x))).collect()


+----------+-----+
|prediction|count|
+----------+-----+
|         1|   23|
|         3|   30|
|         2|   47|
|         0|   50|
+----------+-----+



[(0, 47), (1, 23), (2, 50), (3, 30)]